In [13]:
from bs4 import BeautifulSoup
import selenium
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
import pandas as pd
import numpy as np
import time
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import re
from selenium.common.exceptions import NoSuchElementException
from selenium.common.exceptions import TimeoutException
print(selenium.__version__)

4.8.2


In [14]:
options = webdriver.ChromeOptions()
driver_path = '/Users/kimjisu/Desktop/gwangjin/chromedriver'
driver = webdriver.Chrome(executable_path=driver_path)

/var/folders/m4/3gg6d6z5629g_mxdwwdkylhh0000gn/T/ipykernel_14037/1130565662.py:3: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(executable_path=driver_path)


SessionNotCreatedException: Message: session not created: Chrome failed to start: exited normally.
  (session not created: DevToolsActivePort file doesn't exist)
  (The process started from chrome location /Applications/Google Chrome.app/Contents/MacOS/Google Chrome is no longer running, so ChromeDriver is assuming that Chrome has crashed.)
Stacktrace:
0   chromedriver                        0x0000000102548474 chromedriver + 4326516
1   chromedriver                        0x000000010254093c chromedriver + 4294972
2   chromedriver                        0x000000010216c088 chromedriver + 278664
3   chromedriver                        0x000000010219ba68 chromedriver + 473704
4   chromedriver                        0x0000000102198074 chromedriver + 458868
5   chromedriver                        0x00000001021d7128 chromedriver + 717096
6   chromedriver                        0x00000001021a34e4 chromedriver + 505060
7   chromedriver                        0x00000001021a3f5c chromedriver + 50

In [3]:
value = '광진구 군자동 149-6'

In [4]:
url = 'https://www.yogiyo.co.kr/mobile/#/'
driver.get(url)
time.sleep(2)

# 검색창 선택
search_input = driver.find_element(By.XPATH, '//*[@id="search"]/div/form/input')
search_input.click()
search_input.clear()
time.sleep(0.5)

# 검색창 입력
search_input.send_keys(value)
time.sleep(0.5)

# 검색 버튼 클릭
search_button = driver.find_element(By.XPATH, '//*[@id="button_search_address"]/button[2]')
search_button.click()
time.sleep(1.5)

# 최상위 검색값 클릭
click_first = driver.find_element(By.XPATH, '//*[@id="search"]/div/form/ul/li[3]/a')
click_first.click()

In [5]:
# 스크롤 끝까지
interval = 1
prev_height = driver.execute_script("return document.body.scrollHeight")

    # 반복 수행
while True:
    # 스크롤을 가장 아래로 내림
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight)")

    # 페이지 로딩 대기
    time.sleep(interval)

    # 현재 문서 높이를 가져와서 저장
    curr_height = driver.execute_script("return document.body.scrollHeight")
    if curr_height == prev_height:
        break

    prev_height = curr_height


# 전체 상점 개수 추출
soup = BeautifulSoup(driver.page_source, 'lxml')

store_list = []
stores = driver.find_elements(By.CLASS_NAME, 'col-sm-6')
stores_num = len(list(set(soup.find_all('div', class_='restaurant-name ng-binding'))))
for s in stores:
    store_list.append(s)
stores_num = len(list(set(stores)))
print('stores_num : ', stores_num)

stores_num :  1671


## **크롤링 구간**

### **가게 메뉴 및 정보 크롤링** ###

#### 초기 가게정보

In [ ]:
start = 0

store_data = []
end = stores_num
num = start

while num < end:
    try:
        in_store = driver.find_element(By.XPATH, f'//*[@id="content"]/div/div[4]/div/div[2]/div[{num+1}]/div/table/tbody/tr/td[2]')
        driver.execute_script("arguments[0].click();", in_store)
        time.sleep(0.5)
        soup = BeautifulSoup(driver.page_source, 'lxml')

    except (NoSuchElementException, TimeoutException):
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        WebDriverWait(driver, 10).until(
            lambda driver: driver.execute_script("return document.readyState") == "complete"
        )
        continue
    
    address = soup.find_all('span', class_='tc ng-binding')[3].text.strip()

    if '광진구' not in address:
        driver.back()
    else:
        store_name = soup.find('span', class_='restaurant-name ng-binding').text.strip()
        openhr = soup.find_all('span', class_='tc ng-binding')[1].text.strip()
        mean_rate_div = soup.find('div', class_='restaurant-star-point')
        mean_rate = mean_rate_div.find('strong').text.strip()
        current_url = driver.current_url
        menu_elements = soup.find_all('ul', class_='sub-list')
        
        for parent_element in menu_elements:
            data_rows = parent_element.find_all('div', class_='menu-name ng-binding')
            
            for row in data_rows:
                menu_name = row.text.strip()
                parent_row = row.find_parent('tr')
                photo_exists = 0 if not parent_row or ('ng-hide' in parent_row.get('class', [])) else 1

                store_data.append({
                    'store': store_name,
                    'address': address,
                    'url': current_url, 
                    'openhr': openhr,
                    'menu': menu_name,
                    'photo_exists': photo_exists,
                    'mean_rate': mean_rate
                })
        print(f'{num}. {store_name} 크롤링 완료')
        driver.back()
        time.sleep(0.5)

    num += 1  # 다음 가게로 이동

# DataFrame으로 변환 및 중복 제거
store_df = pd.DataFrame(store_data).drop_duplicates().reset_index(drop=True)
store_df

In [10]:
df = pd.read_csv('dataset/store_data_0405.csv', index_col=0)
store_list = df['store'].unique().tolist()
store_list

['동성관-자양점',
 '24시채린',
 '바로덮밥-건대점',
 '백억커피-화양점',
 '부어치킨-자양고점',
 '뜨닭(찜닭&닭갈비)',
 '아침점심저녁-건대점',
 '배달삼겹돼지되지-구의점',
 '육회바른연어-화양점',
 '라이라이스볶음밥&돈까스-건대점',
 '홍콩반점0410-아차산역점',
 'B사감김밥&닭강정',
 '메이-광진점',
 '노랑통닭-건대점',
 '춘리마라탕-구의점',
 '귀한족발-직영점',
 'BHC-자양3동점',
 '호식이두마리치킨-구의중앙점',
 '지코바치킨-중곡1호점',
 '교촌치킨-구의역점',
 '호식이두마리치킨-화양점',
 '송돼지냉면-건대점',
 '짜글이의 신-건대본점',
 'KFC-세종대점',
 '파스타예요-광진건대본점',
 '청년피자-구의직영점',
 '홍콩반점0410-건대입구역점',
 '춘리마라탕-건대점',
 '꾸브라꼬숯불두마리치킨-자양점',
 '곱도리탕전문점-광진점',
 '장수칡냉면&명동칼국수&찜닭',
 '육회한연어-건대점',
 '맘스터치-건대로데오점',
 '대한칡냉면-광진점',
 '화양동한식밥상',
 '상상오리&삼겹 구의점',
 '본가칼국수',
 '지랄닭발-광진점',
 '굽네치킨&피자-화양점',
 '피자나라치킨공주(자양점)',
 '혼술식당-중곡점',
 '버거킹-군자능동점',
 '삼청동김치찜',
 '호니도니돈카츠&메밀-자양성수점',
 '롯데리아-구의역점',
 '카츠와이찌-중곡점',
 '김치찜에진심-건대점',
 '동궁찜닭-건대광진점',
 '기영이숯불두마리치킨-광진자양점',
 '동대문엽기떡볶이-건대2호직영점',
 '두마리찜닭두찜-건대점',
 '버거킹-건대입구역점',
 'NEWLA수제돈까스',
 '처갓집양념치킨-중곡역점',
 '덮밥외길인생-자양점',
 '국민우유집-건대점',
 '동대문엽기떡볶이-자양점',
 '프랭크수제치즈버거-건대1호점',
 '맘스터치피자앤치킨-건대점',
 '바른죽',
 '롯데리아-아차산역점',
 '프랑킨숯불양념구이치킨-건대역점',
 '동대문엽기떡볶이-아차산역점',
 '도미노피자-군자점',
 '육초연육회초밥연어',
 

#### 중복 제거 크롤링

In [8]:
start = 0
store_data = []

end = stores_num
num = start

while num < end:
    try:
        in_store = driver.find_element(By.XPATH, f'//*[@id="content"]/div/div[4]/div/div[2]/div[{num+1}]/div/table/tbody/tr/td[2]')
        driver.execute_script("arguments[0].click();", in_store)
        time.sleep(0.5)
        soup = BeautifulSoup(driver.page_source, 'lxml')

    except (NoSuchElementException, TimeoutException):
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        WebDriverWait(driver, 10).until(
            lambda driver: driver.execute_script("return document.readyState") == "complete"
        )
        continue
    
    address = soup.find_all('span', class_='tc ng-binding')[3].text.strip()

    if '광진구' not in address:
        driver.back()
    else:

        store_name = soup.find('span', class_='restaurant-name ng-binding').text.strip()
        
        # 여기서 store_name이 이미 크롤링된 리스트에 있는지 확인
        if store_name in store_list:
            print(f'{num}. {store_name}은(는) 이미 리스트에 있습니다. 크롤링을 건너뜁니다.')
            driver.back()
            time.sleep(0.5)
            num += 1
            continue

        store_name = soup.find('span', class_='restaurant-name ng-binding').text.strip()
        openhr = soup.find_all('span', class_='tc ng-binding')[1].text.strip()
        mean_rate_div = soup.find('div', class_='restaurant-star-point')
        mean_rate = mean_rate_div.find('strong').text.strip()
        current_url = driver.current_url
        menu_elements = soup.find_all('ul', class_='sub-list')
        
        for parent_element in menu_elements:
            data_rows = parent_element.find_all('div', class_='menu-name ng-binding')
            
            for row in data_rows:
                menu_name = row.text.strip()
                parent_row = row.find_parent('tr')
                photo_exists = 0 if not parent_row or ('ng-hide' in parent_row.get('class', [])) else 1

                store_data.append({
                    'store': store_name,
                    'address': address,
                    'url': current_url, 
                    'openhr': openhr,
                    'menu': menu_name,
                    'photo_exists': photo_exists,
                    'mean_rate': mean_rate
                })
        print(f'{num}. {store_name} 크롤링 완료')
        driver.back()
        time.sleep(0.5)

    num += 1  # 다음 가게로 이동

# DataFrame으로 변환 및 중복 제거
store_df = pd.DataFrame(store_data).drop_duplicates().reset_index(drop=True)
store_df

422. 주하객잔-구의점은(는) 이미 리스트에 있습니다. 크롤링을 건너뜁니다.
423. 버킷바베큐 크롤링 완료
426. 인생면가은(는) 이미 리스트에 있습니다. 크롤링을 건너뜁니다.
427. 인생면가은(는) 이미 리스트에 있습니다. 크롤링을 건너뜁니다.
428. 육회2990-군자점은(는) 이미 리스트에 있습니다. 크롤링을 건너뜁니다.
432. 카페다비다-핸드드립&더치&수제샌드위치 크롤링 완료
433. 빙달-건대점 크롤링 완료
434. BHC-중곡대박점은(는) 이미 리스트에 있습니다. 크롤링을 건너뜁니다.
435. 지코바치킨-화양점은(는) 이미 리스트에 있습니다. 크롤링을 건너뜁니다.
436. 춘리마라탕-아차산역점 크롤링 완료
437. 옐로우피자-건대점은(는) 이미 리스트에 있습니다. 크롤링을 건너뜁니다.
440. 불맛오지네은(는) 이미 리스트에 있습니다. 크롤링을 건너뜁니다.
442. 배떡-광장점은(는) 이미 리스트에 있습니다. 크롤링을 건너뜁니다.
444. 크로와상점-건대점은(는) 이미 리스트에 있습니다. 크롤링을 건너뜁니다.
446. 훌랄라참숯치킨-화양세종대점 크롤링 완료
447. 초밥이야은(는) 이미 리스트에 있습니다. 크롤링을 건너뜁니다.
448. 갱상도야채고기찜-건대점은(는) 이미 리스트에 있습니다. 크롤링을 건너뜁니다.
452. 건대황제참치은(는) 이미 리스트에 있습니다. 크롤링을 건너뜁니다.
453. 마니달오프리미엄과일전문-건대점 크롤링 완료
454. 닥엔돈스쪽갈비-건대점은(는) 이미 리스트에 있습니다. 크롤링을 건너뜁니다.
455. 팔당왕족발 크롤링 완료
456. 백종원의한신포차닭발&통닭-구의역점은(는) 이미 리스트에 있습니다. 크롤링을 건너뜁니다.
457. 군자대한곱창-군자본점은(는) 이미 리스트에 있습니다. 크롤링을 건너뜁니다.
460. 아이러브피자-구의점은(는) 이미 리스트에 있습니다. 크롤링을 건너뜁니다.
461. 도른계은(는) 이미 리스트에 있습니다. 크롤링을 건너뜁니다.
462. 응급실국물떡볶이&튀김-자양점은(는) 이미 리스트에 있습니

WebDriverException: Message: unknown error: session deleted because of page crash
from unknown error: cannot determine loading status
from tab crashed
  (Session info: chrome=123.0.6312.107)
Stacktrace:
0   chromedriver                        0x0000000100da4474 chromedriver + 4326516
1   chromedriver                        0x0000000100d9c93c chromedriver + 4294972
2   chromedriver                        0x00000001009c7eec chromedriver + 278252
3   chromedriver                        0x00000001009b1fac chromedriver + 188332
4   chromedriver                        0x00000001009afe60 chromedriver + 179808
5   chromedriver                        0x00000001009b0930 chromedriver + 182576
6   chromedriver                        0x00000001009bf3b0 chromedriver + 242608
7   chromedriver                        0x0000000100a4337c chromedriver + 783228
8   chromedriver                        0x00000001009ff4e4 chromedriver + 505060
9   chromedriver                        0x00000001009fff5c chromed

In [9]:
store_df = pd.DataFrame(store_data).drop_duplicates().reset_index(drop=True)
store_df.to_csv('dataset/0405_군자동(2).csv')
store_df

,store,address,url,openhr,menu,photo_exists,mean_rate
0,버킷바베큐,서울특별시 광진구 자양동 619-24 1층(자양동),https://www.yogiyo.co.kr/mobile/#/1236017/,12:00 - 02:00,[둘이서 대접받는 한끼] 올버킷 바베큐（M）,1,5.0
1,버킷바베큐,서울특별시 광진구 자양동 619-24 1층(자양동),https://www.yogiyo.co.kr/mobile/#/1236017/,12:00 - 02:00,[배달업계 최초 미슐랭출신 사장님] 올버킷 바베큐（L）,1,5.0
2,버킷바베큐,서울특별시 광진구 자양동 619-24 1층(자양동),https://www.yogiyo.co.kr/mobile/#/1236017/,12:00 - 02:00,[1인도 특별해요]오리 바베큐 도시락,1,5.0
3,버킷바베큐,서울특별시 광진구 자양동 619-24 1층(자양동),https://www.yogiyo.co.kr/mobile/#/1236017/,12:00 - 02:00,[혼밥도 우아하게]치킨 바베큐 도시락,1,5.0
4,버킷바베큐,서울특별시 광진구 자양동 619-24 1층(자양동),https://www.yogiyo.co.kr/mobile/#/1236017/,12:00 - 02:00,[버킷 시크릿메뉴]핫윙 바베큐 1인,1,5.0
...,...,...,...,...,...,...,...
2259,닥엔돈스직화바베큐치킨-건대점,서울특별시 광진구 구의동 225-56 1층 101호,https://www.yogiyo.co.kr/mobile/#/1201953/,17:00 - 01:30,콜라 500mL,1,5.0
2260,닥엔돈스직화바베큐치킨-건대점,서울특별시 광진구 구의동 225-56 1층 101호,https://www.yogiyo.co.kr/mobile/#/1201953/,17:00 - 01:30,콜라 1.25L,1,5.0
2261,닥엔돈스직화바베큐치킨-건대점,서울특별시 광진구 구의동 225-56 1층 101호,https://www.yogiyo.co.kr/mobile/#/1201953/,17:00 - 01:30,제로콜라,1,5.0
2262,닥엔돈스직화바베큐치킨-건대점,서울특별시 광진구 구의동 225-56 1층 101호,https://www.yogiyo.co.kr/mobile/#/1201953/,17:00 - 01:30,칠성사이다,1,5.0


### 리뷰 데이터 크롤링

In [ ]:
## 가게 정보 테이블 / 리뷰 테이블 분리
store_data = []
review_data = []

for start, end, scroll_count in scroll_ranges:
        
    for num in range(start, end):

        for _ in range(scroll_count):
            driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
            time.sleep(2)  # 페이지 로딩 대기

        time.sleep(2)
        in_store = driver.find_element(By.XPATH, f'//*[@id="content"]/div/div[4]/div/div[2]/div[{num+1}]/div/table/tbody/tr/td[2]')

        # 상점 페이지 클릭
        driver.execute_script("arguments[0].click();", in_store)
        time.sleep(2)

        soup = BeautifulSoup(driver.page_source, 'lxml')

        # 가게 기본 데이터
        address = soup.find_all('span', class_='tc ng-binding')[3].text.strip()

        if '광진구' not in address:
            driver.back() # 광진구 이외 데이터를 제거하는 조건문
        else:
            store_name = soup.find('span', class_='restaurant-name ng-binding').text.strip()
            openhr = soup.find_all('span', class_='tc ng-binding')[1].text.strip()
            mean_rate_div = soup.find('div', class_='restaurant-star-point')
            mean_rate = mean_rate_div.find('strong').text.strip()

            # 가게의 요기요 URL 얻기
            current_url = driver.current_url

            # 메뉴 데이터
            menu_elements = soup.find_all('ul', class_='sub-list')

            for parent_element in menu_elements:

                data_rows = parent_element.find_all('div', class_='menu-name ng-binding')

                for idx, row in enumerate(data_rows):
                    menu_name = row.text.strip()
                    parent_row = row.find_parent('tr')
                    
                    if parent_row:
                        # 메뉴 별로 이미지 여부 확인 (1: 있음, 0: 없음)
                        photo_area = parent_row.find('td', class_='photo-area')
                        photo_exists = 0 if photo_area and 'ng-hide' in photo_area.get('class', []) else 1
                    else:
                        photo_exists = 0

                    store_data.append({'store': store_name,
                                    'address': address,
                                    'url': current_url, 
                                    'openhr': openhr,
                                    'menu': menu_name,
                                    'photo_exists': photo_exists,
                                    'url': current_url})
                    
            # 리뷰 데이터
            review_btn = driver.find_element(By.XPATH, '//*[@id="content"]/div[2]/div[1]/ul/li[2]/a')
            review_btn.click()

            time.sleep(1)

            # 더보기 버튼 클릭 함수
            def click_more_button():
                try:
                    driver.execute_script("window.scrollTo(0, document.body.scrollHeight)")
                    plus_btn = WebDriverWait(driver, 10).until(
                        EC.element_to_be_clickable((By.XPATH, plus_btn_xpath))
                    )
                    plus_btn.click()
                    return True
                except:
                    return False

            i = 0
            while True:
                i += 1
                var = i * 10 + 2
                plus_btn_xpath = f'/html/body/div[6]/div[2]/div[1]/div[5]/ul/li[{var}]/a'

                if not click_more_button():
                    break

                if i%50 == 0:
                    print(f"{i*10}번째 리뷰 크롤링 중...")

                time.sleep(1.5)  # 클릭 후 대기 시간 추가

            soup = BeautifulSoup(driver.page_source, 'lxml')
            review_elements = soup.find_all('li', class_="list-group-item star-point ng-scope")

            for review_element in review_elements:
                score = review_element.find('span', class_='category') # 리뷰 데이터 찾기
                categorical_score = score.find_all('span', class_='points ng-binding') # 카테고리별 점수
                taste_score = categorical_score[0].text.strip() # 맛 점수
                quantity_score = categorical_score[1].text.strip() # 양 점수
                review_time = review_element.find('span', class_='review-time ng-binding').text.strip() # 리뷰 등록일

                if len(categorical_score) > 2:
                    delivery_score = categorical_score[2].text.strip() # 배달 점수
                else:
                    delivery_score = None

                review_content = review_element.find('p', class_='ng-binding').text.strip()

                review_data.append({'store_name': store_name,
                                    'address': address,
                                    'taste_score': taste_score,
                                    'quantity_score': quantity_score,
                                    'delivery_score': delivery_score,
                                    'review': review_content,
                                    'review_time': review_time
                                    })

            driver.back()

# 데이터프레임 생성
store_df = pd.DataFrame(store_data).drop_duplicates().reset_index(drop=True)
store_df.to_csv(f'dataset/0405_{value}_가게정보.csv')

review_df = pd.DataFrame(review_data).drop_duplicates().reset_index(drop=True)
review_df.to_csv(f'dataset/0405_{value}_리뷰.csv')